To run this code you need to first create a hugging face account. Please see how to do this [here]

## Install necessary libraries

In [1]:
#!pip install datasets transformers
#!pip install wandb

In [2]:
# Import the load_dataset function from the 'datasets' library for dataset loading
from datasets import load_dataset

# Import AutoTokenizer from the 'transformers' library for handling tokenization
from transformers import AutoTokenizer

# Import Tokenizer, WordLevel model, WhitespaceSplit pre_tokenizer, and WordLevelTrainer from the 'tokenizers' library
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer

# Import PreTrainedTokenizerFast from the 'transformers' library for efficient tokenization
from transformers import PreTrainedTokenizerFast

# Import notebook_login from the 'huggingface_hub' library for logging into the Hugging Face Model Hub
from huggingface_hub import notebook_login

# Import pandas for data manipulation
import pandas as pd

# Import wandb for logging experiments with Weights & Biases
import wandb

In [4]:
# Set parameters for WandB (Weights & Biases) integration
wandb_project = "pop909_musicgen"
entity = "musicgen"
data_processed = "pop909_processed"

## Download Dataset from Hugging Face

In [5]:
# Import necessary library or module for loading datasets
from datasets import load_dataset

# Load the dataset named "aimusicgen/pop909_clean_data" with the "train" split
ds = load_dataset("aimusicgen/pop909_clean_data", split="train")

# Split the loaded dataset into training and testing sets
# Set test_size to 0.1, indicating that 10% of the data will be used for testing
# Shuffle the data to ensure randomness in the selection of training and testing samples
raw_datasets = ds.train_test_split(test_size=0.1, shuffle=True)

# Display the resulting raw datasets, which now consist of training and testing sets
raw_datasets


Found cached dataset parquet (C:/Users/naomi/.cache/huggingface/datasets/aimusicgen___parquet/aimusicgen--pop909_clean_data-8139a41134aae9f9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 29930
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3326
    })
})

## Train the tokenizer

Let's start by seeing how the default GPT-2 tokenizer works on our dataset

In [6]:
# Extract the "text" field from the "train" split of the raw_datasets

# Select the 11th sample (index 10 since indexing starts at 0) from the "train" split
sample_10 = raw_datasets["train"]["text"][10]

# Extract a substring from the selected sample, taking the first 242 characters
sample = sample_10[:242]

# Display the resulting substring (sample)
sample


'PIECE_START TRACK_START INST=0 BAR_START TIME_DELTA=24 NOTE_ON=58 NOTE_ON=70 NOTE_ON=74 TIME_DELTA=3 NOTE_OFF=70 NOTE_OFF=74 TIME_DELTA=6 NOTE_ON=70 NOTE_ON=74 NOTE_ON=58 NOTE_ON=62 NOTE_ON=67 TIME_DELTA=4 NOTE_OFF=70 NOTE_OFF=74 TIME_DELTA=5'

In [7]:
# Load the pre-trained GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Print the tokens obtained by tokenizing the 'sample' using the GPT-2 tokenizer
# The 'sample' is a text string, and the .tokens() method returns the list of tokens
print(tokenizer(sample).tokens())


['PI', 'EC', 'E', '_', 'ST', 'ART', 'ĠTR', 'ACK', '_', 'ST', 'ART', 'ĠINST', '=', '0', 'ĠBAR', '_', 'ST', 'ART', 'ĠTIME', '_', 'D', 'EL', 'TA', '=', '24', 'ĠNOTE', '_', 'ON', '=', '58', 'ĠNOTE', '_', 'ON', '=', '70', 'ĠNOTE', '_', 'ON', '=', '74', 'ĠTIME', '_', 'D', 'EL', 'TA', '=', '3', 'ĠNOTE', '_', 'OFF', '=', '70', 'ĠNOTE', '_', 'OFF', '=', '74', 'ĠTIME', '_', 'D', 'EL', 'TA', '=', '6', 'ĠNOTE', '_', 'ON', '=', '70', 'ĠNOTE', '_', 'ON', '=', '74', 'ĠNOTE', '_', 'ON', '=', '58', 'ĠNOTE', '_', 'ON', '=', '62', 'ĠNOTE', '_', 'ON', '=', '67', 'ĠTIME', '_', 'D', 'EL', 'TA', '=', '4', 'ĠNOTE', '_', 'OFF', '=', '70', 'ĠNOTE', '_', 'OFF', '=', '74', 'ĠTIME', '_', 'D', 'EL', 'TA', '=', '5']


Not the best since it's not familiar with this English vocabulary, the tokenizer is using quite a few subwords. The tokenizer process involves four steps: 

1. Normalization, 
2. Pretokenization, 
3. Applying the tokenizer model, and 
4. Postprocessing. 

Let's break down each step in our example.

### 1. Normalization

In the normalization step, we perform *"general cleanup, such as eliminating unnecessary whitespace, converting to lowercase, and/or removing accents..."* as per the HF course.

Since our vocabulary is already normalized, there's no requirement to eliminate whitespace, convert to lowercase, or perform any additional cleanup. This step can be bypassed.

### 2. Pretokenization

*"As outlined in the upcoming sections, training a tokenizer solely on raw text is not feasible. The initial step involves breaking down the texts into smaller units, such as words. This is where the pre-tokenization step becomes crucial. As demonstrated in Chapter 2, a word-based tokenizer can easily segment raw text into words based on whitespace and punctuation."* HF course.

In our scenario, this step is straightforward; our pretokenization aims to segment our text into "words" since our dataset is already a sequence of tokens. Therefore, a Whitespace pre_tokenizer would be suitable here. Once again, the chosen model is "WordLevel".

In [8]:
# Initialize a new tokenizer with the WordLevel model
# The 'unk_token="[UNK]"' parameter sets the token to be used for unknown or out-of-vocabulary words
new_tokenizer = Tokenizer(model=WordLevel(unk_token="[UNK]"))

In [9]:
# Assign the WhitespaceSplit pre_tokenizer to the 'pre_tokenizer' attribute of the new_tokenizer
new_tokenizer.pre_tokenizer = WhitespaceSplit()

In [10]:
# Let's test our pre_tokenizer
new_tokenizer.pre_tokenizer.pre_tokenize_str(sample)

[('PIECE_START', (0, 11)),
 ('TRACK_START', (12, 23)),
 ('INST=0', (24, 30)),
 ('BAR_START', (31, 40)),
 ('TIME_DELTA=24', (41, 54)),
 ('NOTE_ON=58', (55, 65)),
 ('NOTE_ON=70', (66, 76)),
 ('NOTE_ON=74', (77, 87)),
 ('TIME_DELTA=3', (88, 100)),
 ('NOTE_OFF=70', (101, 112)),
 ('NOTE_OFF=74', (113, 124)),
 ('TIME_DELTA=6', (125, 137)),
 ('NOTE_ON=70', (138, 148)),
 ('NOTE_ON=74', (149, 159)),
 ('NOTE_ON=58', (160, 170)),
 ('NOTE_ON=62', (171, 181)),
 ('NOTE_ON=67', (182, 192)),
 ('TIME_DELTA=4', (193, 205)),
 ('NOTE_OFF=70', (206, 217)),
 ('NOTE_OFF=74', (218, 229)),
 ('TIME_DELTA=5', (230, 242))]

### 3. Tokenizer model training

In [11]:
# This function will yield the samples to train our tokenizer
# Define a function named 'get_training_corpus' to generate training data in chunks
def get_training_corpus():
  # Access the "train" split of the raw_datasets
  dataset = raw_datasets["train"]
  
  # Iterate through the dataset in chunks of 1000 samples
  for i in range(0, len(dataset), 1000):
    # Yield the "text" field of each chunk, creating a generator for training corpus
    yield dataset[i : i + 1000]["text"]

In [12]:
# Initialize a WordLevelTrainer for training a WordLevel tokenizer
# The trainer is configured with special tokens including "[UNK]", "[CLS]", "[SEP]", "[PAD]", and "[MASK]"
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

**[UNK] (Unknown Token):**
Represents unknown or out-of-vocabulary words. When the tokenizer encounters a word not present in its vocabulary, it replaces the unknown word with "[UNK]" during tokenization.Handles words that are not part of the model's training vocabulary, ensuring that even unseen words can be represented.<br>
**[CLS] (Classification Token):**
Represents the beginning of a sequence or a classification task. It is often used in conjunction with sequence classification tasks or sentence-level embeddings. Indicates the start of a sequence, helping models understand the structure of input data and facilitating tasks like sentence classification.<br>
**[SEP] (Separator Token)**
Represents the separation between two segments in a sequence. Commonly used to separate sentences or segments in tasks like question-answering or text generation. Helps the model distinguish between different parts of the input sequence, guiding the model to understand relationships between segments.<br>
**[PAD] (Padding Token)**
Represents padding in sequences. It is used to make sequences of variable lengths equal in size by adding padding tokens to shorter sequences. Ensures that input sequences have consistent lengths, allowing for efficient batch processing during training and inference.<br>
**[MASK] (Mask Token)**
Used in masked language modeling tasks. During training, some words are replaced with "[MASK]" tokens, and the model is tasked with predicting the original words based on the context. Supports the pre-training of language models by training the model to predict missing or masked words, enhancing its ability to understand context and relationships between words.<br>

In [13]:
# Train the 'new_tokenizer' using the training data generated by the 'get_training_corpus' function
# The training is performed with the specified 'trainer', which is a WordLevelTrainer
new_tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

### 4. Post processing and save it to the hub

In [14]:
# Save the trained 'new_tokenizer' to a file named "tokenizer.json"
new_tokenizer.save("tokenizer.json")

# Load the saved tokenizer from "tokenizer.json" and create a new instance of PreTrainedTokenizerFast
new_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")

# Add a special token, '[PAD]', to the loaded tokenizer
new_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# We will receive '0' as output if saving was successful with no errors

0

You must create an 'Access Token' in hugging face with the same name as the new tokenizer to run the below. i.e. pop909_tokenizer.

In [30]:
# Log in to the Hugging Face Model Hub 
notebook_login()

In [31]:
# Push the trained 'new_tokenizer' to the Hugging Face Model Hub 
new_tokenizer.push_to_hub("aimusicgen/pop909_tokenizer")

CommitInfo(commit_url='https://huggingface.co/aimusicgen/pop909_tokenizer/commit/6fe1ee932447b6cfe5e0f5cee3480e405d0136ea', commit_message='Upload tokenizer', commit_description='', oid='6fe1ee932447b6cfe5e0f5cee3480e405d0136ea', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
# Load a pre-trained tokenizer using the AutoTokenizer class from the Hugging Face Model Hub
# The tokenizer is loaded from the repository named "aimusicgen/pop909_tokenizer"
load_tokenizer = AutoTokenizer.from_pretrained("aimusicgen/pop909_tokenizer")

In [33]:
# Tokenize the 'sample' text using the pre-trained 'load_tokenizer'
# The 'sample' text is expected to be processed into a sequence of tokens
load_tokenizer(sample)

{'input_ids': [100, 62, 60, 12, 9, 43, 49, 45, 10, 48, 44, 14, 49, 45, 43, 17, 19, 7, 48, 44, 8], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

What we can see here is the tokenized text in the form of dictionaries.<br>

**input_ids:** Represents the tokenized input sequence where each number corresponds to a specific token.Each number corresponds to a token in the vocabulary.<br>

**token_type_ids:** Represents the segment or sentence to which each token belongs. All tokens in the sequence belong to the same segment or sentence (segment 0). <br>

**attention_mask:** Represents the attention mask indicating which tokens should be attended to (have a value of 1) and which should be ignored (have a value of 0). All tokens in the sequence are attended to, indicating that none should be ignored.

In [34]:
# Tokenize the 'sample' text using the pre-trained 'load_tokenizer'
# Retrieve and display the list of tokens obtained from the tokenization process
load_tokenizer(sample).tokens()

['PIECE_START',
 'TRACK_START',
 'INST=0',
 'BAR_START',
 'TIME_DELTA=24',
 'NOTE_ON=58',
 'NOTE_ON=70',
 'NOTE_ON=74',
 'TIME_DELTA=3',
 'NOTE_OFF=70',
 'NOTE_OFF=74',
 'TIME_DELTA=6',
 'NOTE_ON=70',
 'NOTE_ON=74',
 'NOTE_ON=58',
 'NOTE_ON=62',
 'NOTE_ON=67',
 'TIME_DELTA=4',
 'NOTE_OFF=70',
 'NOTE_OFF=74',
 'TIME_DELTA=5']

In [35]:
# Load a pre-trained tokenizer using the AutoTokenizer class from the Hugging Face Model Hub
# The tokenizer is loaded from the repository named "aimusicgen/pop909_tokenizer"
tokenizer = AutoTokenizer.from_pretrained("aimusicgen/pop909_tokenizer")

# Tokenize the 'sample' text using the loaded tokenizer
# Print and display the list of tokens obtained from the tokenization process
print(tokenizer(sample).tokens())

In [36]:
# Obtain the vocabulary of the tokenizer using the 'get_vocab' method
vocab = tokenizer.get_vocab()

# Display and store the obtained vocabulary
vocab

{'BAR_END': 11,
 'NOTE_OFF=55': 50,
 'NOTE_ON=72': 47,
 'NOTE_ON=60': 25,
 'NOTE_ON=37': 124,
 'NOTE_OFF=84': 103,
 'NOTE_ON=92': 136,
 'NOTE_ON=45': 91,
 'NOTE_ON=51': 83,
 'NOTE_ON=103': 176,
 'NOTE_OFF=48': 80,
 'NOTE_ON=102': 178,
 'NOTE_ON=62': 17,
 'NOTE_ON=30': 146,
 'NOTE_OFF=69': 32,
 '[UNK]': 0,
 'NOTE_ON=76': 53,
 'NOTE_OFF=90': 129,
 'NOTE_OFF=38': 115,
 'TRACK_START': 62,
 'NOTE_ON=73': 55,
 'NOTE_ON=31': 150,
 'NOTE_ON=32': 140,
 'NOTE_ON=28': 154,
 'NOTE_OFF=62': 16,
 'NOTE_OFF=61': 36,
 'PIECE_START': 100,
 'NOTE_OFF=56': 58,
 'NOTE_OFF=92': 135,
 'NOTE_OFF=31': 149,
 'NOTE_OFF=57': 34,
 'NOTE_ON=83': 102,
 '[CLS]': 1,
 'NOTE_ON=61': 37,
 'NOTE_ON=90': 130,
 'NOTE_ON=42': 108,
 'NOTE_ON=105': 180,
 'TIME_DELTA=3': 10,
 'NOTE_OFF=34': 137,
 'NOTE_OFF=102': 177,
 'NOTE_ON=34': 138,
 'NOTE_ON=43': 93,
 'NOTE_ON=85': 114,
 'NOTE_ON=96': 156,
 'NOTE_OFF=77': 63,
 'NOTE_OFF=63': 28,
 'NOTE_ON=84': 104,
 'NOTE_ON=33': 144,
 'NOTE_ON=54': 66,
 'NOTE_OFF=75': 56,
 'NOTE_ON=56': 

The vocabulary of the tokenizer is the equalivant as a metadata dictionary for the tokenizer we created. It maps tokens to numerical indices, providing information about the unique elements (tokens) present in the dataset.It outlines the structure and organization of the tokenization process, detailing how words or subwords are represented by numerical indices.

Create a dataframe with the tokenizer's vocabulary to upload it to weights & bias.

In [37]:
# Create a DataFrame 'df' using a list comprehension, where each row contains a token and its corresponding index from
# the tokenizer's vocabulary
df = pd.DataFrame([{"Token": token, "Index": idx} for token, idx in vocab.items()]).sort_values(by="Index")

# Display the DataFrame 'df', which presents the tokens and their associated indices in the tokenizer's vocabulary
df

,Token,Index
15,[UNK],0
32,[CLS],1
153,[SEP],2
168,[PAD],3
133,[MASK],4
...,...,...
36,NOTE_ON=105,180
103,NOTE_OFF=25,181
104,NOTE_ON=25,182
52,NOTE_OFF=106,183


### Upload vocab to W&B

In [38]:
# Initialize a Weights & Biases (wandb) run with project name is set to "pop909_pretokenization", 
# and the job type is set to "upload"
run = wandb.init(project=wandb_project, job_type="upload")

In [39]:
# Create table with vocab
vocab_table = wandb.Table(data=df)

"If your framework uses or produces models or datasets, you can log them for full traceability and have wandb automatically monitor your entire pipeline through W&B Artifacts." - https://docs.wandb.ai/guides/integrations/add-wandb-to-any-library

In [40]:
# Create artifact for raw data
processed_data_at = wandb.Artifact(name=data_processed, type="processed_data")

In [41]:
# Add 'vocab_table' to 'processed_data_at' artifact with the name "vocab_table"
processed_data_at.add(vocab_table, name="vocab_table")

ArtifactManifestEntry(path='vocab_table.table.json', digest='jGQiS9MBJFnW0TOd0jjl7w==', size=4226, local_path='C:\\Users\\naomi\\AppData\\Local\\wandb\\wandb\\artifacts\\staging\\tmp6lagf24e')

In [42]:
# Log the 'processed_data_at' artifact to the Weights & Biases run
run.log_artifact(processed_data_at)

<Artifact pop909-processed>

In [43]:
# Complete and finish the Weights & Biases run
run.finish()